In [ ]:
#Modusschalter     Analyse aller CSV
!pip install gradio
import gradio as gr
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go
import re
import os
import glob
import io
##########
# 📁 Создание необходимых папок
import os
os.makedirs("parsed/google", exist_ok=True)

# 📤 Загрузка файлов вручную
from google.colab import files
uploaded = files.upload()

import shutil
for fname in uploaded:
    shutil.move(fname, f"parsed/google/{fname}")
#my googledisk in Golab
#from google.colab import drive
#drive.mount('/content/drive')
#import os
# Указываем путь, где будут храниться файлы
#path = "/content/drive/MyDrive/parsed/"
# Проверим, существует ли директория
#if not os.path.exists(path):
#    os.makedirs(path)
# Смотрим, что в ней
print("Содержимое директории 'parsed':", os.listdir("parsed"))               #os.listdir(path))
##from google.colab import files
#uploaded = files.upload()  # Загрузите файлы с вашего устройства                                          # Meine disk c

# Например, файл можно переместить в нужную директорию
#import shutil
#shutil.move('*.csv', '/content/drive/MyDrive/parsed')

############

from google.colab import drive
drive.mount('/content/drive')

#print("Текущая рабочая директория:", os.getcwd())
#glob.glob(r"D:\Downloads\PROEKT_FINAL\proek\parsed\google\*.csv")+ glob.glob("parsed/google/*.CSV")         # Meine disk c
#csv_files = glob.glob(r"D:\Downloads\PROEKT_FINAL\proekt\parsed\google\*.csv")      #, recursive=True
csv_files = glob.glob("/content/drive/MyDrive/parsed/google/*.csv")

print("Найдено файлов:", len(csv_files))
for f in csv_files:
    print("📄", f)

#
# === Шаг 1: словарь компаний ===
company_ticker_map = {
    "apple": "AAPL",
    "microsoft": "MSFT",
    "google": "GOOGL",
    "meta": "META",
    "nvidia": "NVDA"
}
## Укажи полный путь до корня, где находится папка "parsed"
#os.chdir(r"D:\Downloads\PROEKT_FINAL\proekt")
#print("✅ Текущая директория:", os.getcwd())
#print("📁 Содержимое:", os.listdir())

#print("Файлы в parsed/google/:")
#for f in glob.glob("parsed/google/*.csv"):
#    print("📄", f)

import os
print("Содержимое parsed/:", os.listdir("parsed"))

# === Parsen einer Benutzeranforderung (für eine Datei) ===
def parse_request(user_input):
    user_input = user_input.lower()
    ticker = None
    for name in company_ticker_map:
        if name in user_input:
            ticker = company_ticker_map[name]
            company_name = name.title()
            break
    if not ticker:
        raise ValueError("❌ Das Unternehmen kann nicht aus der Abfrage ermittelt werden.")
    days = 90
    match = re.search(r"(\d+)\s*Monat", user_input)
    if match:
        days = int(match.group(1)) * 30
    return ticker, company_name, days

# === Analyse einer CSV ===
def analyze_query(user_input, csv_file):
    try:
        #df = pd.read_csv(io.StringIO(csv_file.decode("utf-8")))                   #  df = pd.read_csv(csv_file.name)
        # Загрузка файла
        df = pd.read_csv(csv_file.name)                                            #pd.read_csv('/content/drive/MyDrive/parsed/*.csv')

       # Вывод первых 5 строк
        print(df.head())
        if not {"Date", "Close"}.issubset(df.columns):
            return go.Figure(), "⚠️ CSV-файл должен содержать колонки 'Date' и 'Close'."
        df["Date"] = pd.to_datetime(df["Date"])
        df = df.dropna()
        ticker, company_name, forecast_days = parse_request(user_input)
        prophet_df = df[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})
        model = Prophet()
        model.fit(prophet_df)
        future = model.make_future_dataframe(periods=forecast_days)
        forecast = model.predict(future)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Preise sind historisch'))
        fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Vorhersage'))
        fig.update_layout(title=f"{company_name} — Vorhersage", xaxis_title='Datum', yaxis_title='Preis')
        latest = forecast['yhat'].iloc[-1]
        earlier = forecast['yhat'].iloc[-forecast_days - 1]
        change = latest - earlier
        change_pct = (change / earlier) * 100
        trend = "Höhe" if change > 0 else "Verringern"
        text = f"🔮 Vorhersage: erwarte {trend} ~{abs(change_pct):.2f}% in {forecast_days} Tagen."
        return fig, text
    except Exception as e:
        return go.Figure(), f"⚠️ Fehler: {str(e)}"

# === Анализ всех CSV из папки ===
def analyze_all_csvs():
    #
      csv_paths = glob.glob("parsed/**/*.csv", recursive=True)               #  #csv/*.csv")
    if not csv_paths:
        return go.Figure(), "❌ Es befinden sich keine CSV-Dateien im Ordner parsed."
   #
    fig = go.Figure()
    summaries = []

    for path in csv_paths:
        try:
            print(f"🔍 Dateiverarbeitung: {path}")
            df = pd.read_csv(path)
            if not {"Date", "Close"}.issubset(df.columns):
                print(f"⚠️ {path}: Keine Lautsprecher 'Date' и 'Close'")
                continue
            df["Date"] = pd.to_datetime(df["Date"],errors='coerce')
            df = df.dropna()
            if df.empty:
                print(f"⚠️ {path}: Nach der Reinigung entleeren"")
                continue

            if len(df["Date"].unique()) < 2:
                print(f"⚠️ {path}: Nicht genügend Zeitpunkte")
                continue
            prophet_df = df[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})
            model = Prophet()
            model.fit(prophet_df)
            future = model.make_future_dataframe(periods=90)
            forecast = model.predict(future)
            company = os.path.basename(path).replace(".csv", "")
            fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name=company))
            change = forecast['yhat'].iloc[-1] - forecast['yhat'].iloc[-91]
            change_pct = (change / forecast['yhat'].iloc[-91]) * 100
            trend = "Höhe" if change > 0 else "Verringern"
            summaries.append(f"{company}: {trend} ~{abs(change_pct):.2f}%")
        except Exception as e:
            print(f"❌ Fehler in Datei {path}: {e}")
            summaries.append(f"{path}: Fehler — {str(e)}")
  #  except Exception as e:
 #   print(f"❌ Fechler in {path}: {e}")

    fig.update_layout(title="📊 Vergleichsanalyse für alle Unternehmen", xaxis_title="Datum", yaxis_title="Preis")
    return fig, "\n".join(summaries)

# === Управляющая функция ===
def route_query(mode, user_input=None, file=None):
    if mode == "eine Datei":
#         return analyze_query(user_input, file)
         if file:
            return analyze_query(user_input, file)
         else:
            return go.Figure(), "⚠️ Datum Bitte laden Sie die CSV-Datei hoch."
    elif mode == "Все файлы":
         return analyze_all_csvs()
    else:
         return go.Figure(), "⚠️ Bitte wählen Sie einen Analysemodus."

# === Интерфейс Gradio ===
iface = gr.Interface(
    fn=route_query,
    inputs=[
        gr.Radio(["Alle Dateien"], label="Analysemodus"),
        gr.Textbox(label="Anfrage (zum Beispiel: 'Nvidia 3-Monats-Prognose')"),
        gr.File(label="CSV-Datei mit Spalten 'Date' и 'Close'", file_types=[".csv"])
    ],
    outputs=[
        gr.Plot(label="📈 Grafik"),
        gr.Text(label="🧠 Prognosetext")
    ],
    title="📊 Prognosetext Agent für Aktienanalyse und Prognose",
    description="Laden Sie eine einzelne CSV-Datei hoch oder wählen Sie die automatische Analyse aller Dateien in einem Ordner 'parsed/''."
)

iface.launch()
